## gensim으로 네이버 기사 토픽 모델링 해보기

> 토픽 모델링을 적용하기 위해 텍스트를 처리합니다.

> 토픽 모델링 라이브러리인 gensim을 사용해봅니다.

### 1. 토픽 모델링을 위한 라이브러리 불러오기

In [1]:
from tqdm import tqdm_notebook # progress bar
import MeCab # Mecab 형태소 분석기 불러오기
import numpy as np
import string # 특수문자 - 클리닝과정에서 제거하기 위함
import re
import warnings # 경고 알림 제거
import pickle

# gensim에서 사용하는 vectorizer 모듈과, LDA module 로드
from gensim import corpora # LDA를 돌릴 수 있게 형변환 해주는 역할
from gensim import models # includ LDA

#시각화
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning) # 경고 알림이 뜨면 모두 무시합니다.

### MeCab 윈도우용 함수 설정

In [2]:
"""
    Parsing 규칙의 문제점, split을 "," 기준으로 하는데, token이 "," 인 경우에는 쉼표만 잘려서 나오기 때문에, 
    + "%," 같이 특수문자와 쉼표가 같이 등장하는 경우도 생각해주어야 함.
    
    (",", "SC") 의 원래 튜플이 만들어지지 않음.
    
    명사 분석의 경우 해당 토큰이 필요하지 않으니 pass
    
    형태소 분석과 POS tagging의 경우 해당 토큰이 필요하므로, token[0]이 ' 인 경우엔 따로 (",", "SC")를 집어 넣어줘야함.
"""
import MeCab # 윈도우 명령어
import re

mecab = MeCab.Tagger()

def mecab_nouns(text):
    nouns = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    # 추출한 token중에 POS가 명사 분류에 속하는 토큰만 선택.
    for token in temp:
        if token[1] == "NNG" or token[1] == "NNP" or token[1] == "NNB" or token[1] == "NNBC" or token[1] == "NP" or token[1] == "NR":
            nouns.append(token[0])
        
    return nouns

def mecab_morphs(text):
    morphs = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    # 추출한 token중에 문자열만 선택.
    for token in temp:
        morphs.append(token[0])
    
    return morphs

def mecab_pos(text):
    pos = []
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    pos = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    return pos

### 2. 텍스트 전처리 함수 만들기

- 토픽모델링에 들어갈 문서는 1차원 리스트 [문서1, 문서2, 문서3...] 이어야 한다.

In [3]:
def read_documents(input_file_name):
    """문서들을 주어진 이름의 파일로부터 읽어들여 돌려준다."""
    
    corpus = []
    
    # pk파일 불러오는 함수
    with open(input_file_name, 'rb') as f:
        temp_corpus = pickle.load(f)
        
    for page in temp_corpus:
        corpus += page
    
    return corpus

def text_cleaning(docs):
    # 한국어를 제외한 글자를 제거하는 함수.
    cleaned_docs = []
    
    for doc in docs:
        temp_doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
        cleaned_docs.append(temp_doc)

    return cleaned_docs

def define_stopwords(path):
    
    SW = set()
    # 불용어를 추가하는 방법 1.
    # python에 내장된 string에서 제공하는 특수문자를 입력
    for i in string.punctuation:
        SW.add(i)

    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path) as f:
        for word in f:
            SW.add(word)

    return SW


def text_tokenizing(corpus, tokenizer):
    # 명사 추출 / 형태소 분석 두가지를 선택할 수 있도록 하는 함수
    mecab = MeCab.Tagger()
    token_corpus = []
    
    # 명사 추출
    # 웹스크렙핑해서 지저분한 데이터인 경우 사용
    # 명사가 가장 많은 의미를 가지고 있기 때문
    if tokenizer == "noun":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab_nouns(corpus[n])
            # 필터링
            # 추출한 명사에서 불용어인 것 & 단어수가 1개 이상인 것들만 drop
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            
            token_corpus.append(token_text)
    
    # 형태소 분석
    elif tokenizer == "morph":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab_morphs(corpus[n])
            # 추출한 명사에서 불용어인 것 & 단어수가 1개 이상인 것들만 drop
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
    
    # 단어 단위로 토큰화 하기
    elif tokenizer == "word":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = corpus[n].split() # corpus를 split
            # 추출한 명사에서 불용어인 것 & 단어수가 1개 이상인 것들만 drop
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
        

    return token_corpus

# 분석할 문서의 pk파일 (네이버 기사 - 네이버 기사 크롤러 만들기.ipynb)에서 만든 파일로 수집한 pk파일
input_file_name = "naver_news_content.pk"
documents = read_documents(input_file_name)

# 불용어 저장해둔 txt
SW = define_stopwords("stopwords-ko.txt")

# 데이터 클리닝
cleaned_text = text_cleaning(documents)

# 데이터 토큰화 - 명사 단위
tokenized_text = text_tokenizing(cleaned_text, tokenizer="morph") #tokenizer= "noun" or "word" or "morph"

문서 읽기의 과정은 앞서 단어 임베딩의 경우와 다르지 않다. 다음 과정은 문서-단어 행렬을 만드는 과정이다.

In [6]:
# list형태로 이뤄진 단어의 집합 = BOW
# tokenized_text 는 list of list of word 라 할 수 있다.
print(tokenized_text[0])

['본문', '내용', '플레이어', '플레이어', '오류', '우회', '위한', '함수', '추가', '디지털', '데일리', '세아', '기자', '기업', '내부', '업무', '에서', '데이터', '분석', '대한', '요구사항', '면서', '데이터', '분석가', '부족', '데이터', '입력', '위한', '비용', '증가', '어려움', '커지', '한국', '오라클', '이런', '상황', '에서', '기업', '자동', '분석', '이용', '데이터', '분석', '들어가', '비용', '인력', '절감', '머신', '러닝', '서비스', '소개', '오라클', '머신', '러닝', '알고리즘', '대한', '이해', '더라도', '머신', '러닝', '가능', '다고', '강조', '디지털', '데일리', '주최', '클릭', '으로', '가능', '머신', '러닝', '비나', '에서', '오라클', '개발', '언어', '몰라도', '클릭', '기반', '으로', '다양', '산업', '필요', '머신', '러닝', '관련', '업무', '예측', '수행', '자사', '머신', '러닝', '대해', '공유', '시간', '가졌', '자리', '에서', '오라클', '코딩', '개발', '못해', '머신', '러닝', '활용', '다는', '개념', '만들', '집중', '먼저', '오라클', '자사', '추구', '머신', '러닝', '기술', '대한', '아젠다', '공유', '한국', '오라클', '박소희', '부장', '코딩', '해도', '개발', '해도', '러신', '가능', '다는', '전제', '바탕', '으로', '머신', '러닝', '대한', '기본', '개념', '부터', '소개', '부장', '우선', '미국', '머신', '러닝', '대가', '아서', '사무엘', '정의', '차용', '머신', '러닝', '컴퓨터', '공학', '분야', '통계', '기법', '사용', '명백', '프로그램', '아닌',

### 3. 토픽 모델링에 사용할 함수들 확인하기

- 토픽 모델링(LDA사용)의 gensim에 반드시 들어가야 하는 input 2가지
    - corpus
    - dictionary
    
    ---
- 토픽 모델링에 input을 하기 위해서는 BOW(Bag of Word)형태로 만들어야 한다.
    - 이때 사용하는 것이 dictionary
    - python 의 dict로 따지자면 key값이 숫자, value가 각 단어인 형태
    - 컴퓨터에게 각 단어의 위치를 효율적으로 알려주기 위함
   

In [7]:
# 문서-단어 행렬 만들기
# 어휘(vocabulary) 학습 (dictionary 만들기) -> BOW
dictionary = corpora.Dictionary(tokenized_text)

# 문서-단어 행렬(document-term matrix) 생성
# corpus 만들기 (doc2bow 함수 사용)
# sklearn의 CountVectorizer와 동일
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

In [8]:
# 결과 확인
print(dictionary)

Dictionary(1506 unique tokens: ['가능', '가설', '가졌', '가지', '강조']...)


In [6]:
# 숫자로 표현된 것 = BOW로 표현된 것
# (인덱스, 단어의 frequency)
corpus[0][:5]

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1)]

In [9]:
# BOW가 아닌 TFIDF로 활용하고 싶다면?
# TFIDF 문서-단어 행렬 생성 (models.TfidfModel 함수 사용)
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

# BOW로 만든 corpus의 형태와 동일
# 괄호내의 소수점은 TFID값
corpus_tfidf[0][:5]

[(0, 0.17250675676517552),
 (1, 0.02875112612752925),
 (2, 0.04200095094102228),
 (3, 0.02100047547051114),
 (4, 0.02875112612752925)]

In [10]:
# LDA 모델 선언
# corpus(BOW)
# num_topics : 몇개의 토픽으로 나눌 것인가
# id2word : corpus의 어디에 무슨단어가 알수 있도록 가이드 해주는 것
# 가이드 값은 이미 dictionary에서 입력해 둠 

model = models.ldamodel.LdaModel(corpus, num_topics=4, id2word=dictionary)

In [9]:
# 결과 확인
# model.show_topic(topic_#, number of words)

# 3번 토픽의 top10개를 출력해라!
model.show_topic(3, 10)

[('에듀', 0.03548237),
 ('블록', 0.03131921),
 ('교육', 0.028588695),
 ('패키지', 0.02640994),
 ('초급', 0.024832888),
 ('코딩', 0.020167248),
 ('출시', 0.016783467),
 ('지니', 0.015812188),
 ('초등학생', 0.013613544),
 ('가능', 0.013063768)]

- 위와 같이 단어별 중요도가 출력되면 문서내에서 어떤 단어가 중요한지 파악할 수 있다.

### 4. 토픽 모델링을 추가하여 코드 완성하기

In [11]:
# 토픽 개수(=k), 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 3 

NUM_TOPIC_WORDS = 30


# dict와 corpus 입력해주는 함수
def build_doc_term_mat(documents):
    # 문서-단어 행렬 만들어주는 함수.
    print("Building document-term matrix.")
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary


def print_topic_words(model):
    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")
    
    # topic별로 결과를 출력
    # LDA된 결과 
    # NUM_TOPICS 가 3이므로 range도 3이된다.
    for topic_id in range(model.num_topics):
        # 결과 출력
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        # show topic의 결과를 단어별로 1개씩 
        for topic_word, prob in topic_word_probs:
            # tap을 2번 적용
            print("\t{}\t{}".format(topic_word, prob))
            # topic별로 2줄 띄우기
        print("\n")

# document-term matrix를 만들고, document입력(=tokenized_text) 
corpus, dictionary = build_doc_term_mat(tokenized_text)

# LDA를 실행.
# alpha : 
# eta : 
# passes : 몇번 돌릴건지 (문서 많다면 보통 1000번)
model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
models.ldamodel.LdaModel()
# 결과를 출력.
print_topic_words(model)

Building document-term matrix.

Printing topic words.

Topic ID: 0
	교육	0.015427344478666782
	에서	0.01414144691079855
	으로	0.010204826481640339
	대학	0.008172360248863697
	인공지능	0.005609037354588509
	학생	0.005418418440967798
	학교	0.004718398675322533
	과정	0.004470916464924812
	위한	0.004307625815272331
	진행	0.0043004523031413555
	지원	0.0041030775755643845
	수업	0.003932578023523092
	서울	0.003910733852535486
	어요	0.0038768756203353405
	습니다	0.0036684381775557995
	스쿨	0.0036488589830696583
	활용	0.0035630962811410427
	클래스	0.0034752702340483665
	공부	0.00339215318672359
	데이터	0.0033686310052871704
	미네르바	0.0033132818061858416
	총장	0.003307714592665434
	혁신	0.003291135421022773
	까지	0.0032217029947787523
	내용	0.0031743317376822233
	교수	0.003139260457828641
	기술	0.003134637139737606
	배우	0.00309071340598166
	통해	0.0030791168101131916
	온라인	0.0030513748060911894


Topic ID: 1
	에서	0.015744494274258614
	으로	0.0147038409486413
	교육	0.011719219386577606
	대학	0.007059231400489807
	학교	0.007040075492113829
	미네르바	0.006236819550395012
	

### 5. pyLDAvis를 통한 토픽 모델링 결과 시각화하기

In [15]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

# pyLDAvis 실행.
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data # without print()

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
2     -0.017790  0.0       1        1  43.450506
1      0.003892  0.0       2        1  32.754287
0      0.013898  0.0       3        1  23.795207, topic_info=      Term       Freq      Total Category  logprob  loglift
117     에서  57.000000  57.000000  Default  30.0000  30.0000
61      머신  26.000000  26.000000  Default  29.0000  29.0000
203     교육  56.000000  56.000000  Default  28.0000  28.0000
523     학교  20.000000  20.000000  Default  27.0000  27.0000
1154  미네르바  16.000000  16.000000  Default  26.0000  26.0000
...    ...        ...        ...      ...      ...      ...
1154  미네르바   3.532042  16.734471   Topic3  -5.7098  -0.1199
473     보다   3.201292  12.453854   Topic3  -5.8081   0.0772
202     활용   3.798351  25.985933   Topic3  -5.6371  -0.4873
30      내용   3.383918  17.267983   Topic3  -5.7527  -0.1941
45     데이터   3.591046  32.720550   Topic3  -5.6932  -0.7739

[228 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.743534   가능
0         2  0.297413   가능
0         3  0.148707   가능
531       1  0.483967   가치
531       2  0.483967   가치
...     ...       ...  ...
202       2  0.192412   활용
202       3  0.153929   활용
1501      1  0.462984   후반
1501      2  0.462984   후반
1501      3  0.462984   후반

[409 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

- 파란색 bar : 각 토픽내에서 해당 단어의 frequency
- 원형 : multidimensional scaling
    - PCA처럼 고차원의 document를 2차원으로 표현한 것
    - 각 토픽의 크기(단어개수) 별로 비교해서 단어별 BOW(위치) 를 시각화 한 것
    - 같은 x축선에 비슷한 위치에 있다. = 주로 겹치는 내용이 존재 한다. (e.g : 3토픽 모두 '교육' 이 상위권)
- 단어들의 순위가 비슷한 이유?
    - document자체가 적기 때문
